LinearRegression_Phase6_การทดลองที่1_Recursive  
ref : https://towardsdatascience.com/6-methods-for-multi-step-forecasting-823cbde4127a/

#Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv")
df.head()

In [ ]:
df.columns

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

In [ ]:
df['is_weekend'] = (df.index.dayofweek == 6).astype(int)
df.head()

ตัวแปรที่ไม่ได้มีความหมายเชิงลำดับเช่น dayofweek, month  
เพราะตัวแปรเหล่านี้เป็นข้อมูลเชิงกลุ่ม (Categorical) ไม่ใช่เชิงลำดับ (Ordinal) หากเราใส่เป็นตัวเลข 1-12 เข้าไปตรงๆ โมเดลจะเข้าใจผิดว่า "เดือนธันวาคม (12)" มีค่ามากกว่า "เดือนมกราคม (1)" ถึง 12 เท่า ซึ่งไม่เป็นความจริง การทำ One-Hot Encoding จะสร้างคอลัมน์ใหม่เป็น is_Jan, is_Feb,... เพื่อแก้ปัญหานี้

In [ ]:
df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df['is_Mon'] = (df.index.dayofweek == 0).astype(int)
df['is_Tue'] = (df.index.dayofweek == 1).astype(int)
df['is_Wed'] = (df.index.dayofweek == 2).astype(int)
df['is_Thu'] = (df.index.dayofweek == 3).astype(int)
df['is_Fri'] = (df.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df['is_Jan'] = (df.index.month == 1).astype(int)
df['is_Feb'] = (df.index.month == 2).astype(int)
df['is_Mar'] = (df.index.month == 3).astype(int)
df['is_Apr'] = (df.index.month == 4).astype(int)
df['is_May'] = (df.index.month == 5).astype(int)
df['is_Jun'] = (df.index.month == 6).astype(int)
df['is_Jul'] = (df.index.month == 7).astype(int)
df['is_Aug'] = (df.index.month == 8).astype(int)
df['is_Sep'] = (df.index.month == 9).astype(int)
df['is_Oct'] = (df.index.month == 10).astype(int)
df['is_Nov'] = (df.index.month == 11).astype(int)
df['is_Dec'] = (df.index.month == 12).astype(int)
df.head(3)

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df = df.reindex(columns=['sales_sum', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                         , 'year', 'time_index'
                         #, 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'ismarketday', 'isschoolday', 'holiday'
                         ])

df.head()

In [ ]:
# prompt: drop row that have missing value

df.dropna(inplace=True)
df.head()

In [ ]:
df_for_split = df.copy()
train_data = df_for_split.head(120)
test_data = df_for_split.tail(7)
print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

In [ ]:
features_recursive = [
        'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7',
        'year', 'time_index'
        #, 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
        #, 'ismarketday', 'isschoolday', 'holiday'
    ]

target = 'sales_sum'

X_train = train_data[features_recursive]
y_train = train_data[target]

#Train Model

In [ ]:
model_recursive = LinearRegression()
model_recursive.fit(X_train, y_train)
print("\nRecursive model trained successfully.")
print("-" * 50)

In [ ]:
#ทำนายแบบ Recursive 7 ขั้นตอน ---
last_known_lags = train_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']].tail(1).values.flatten()

future_dates = pd.date_range(start=test_data.index[0], periods=7, freq='D')
future_df = pd.DataFrame(index=future_dates)
future_df['year'] = future_df.index.year
future_df['time_index'] = range(train_data['time_index'].max() + 1, train_data['time_index'].max() + 1 + 7)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

#for i, month_name in enumerate(months):
 #   future_df[f'is_{month_name}'] = (future_df.index.month == i + 1).astype(int)

In [ ]:
print(future_df.shape)
future_df

In [ ]:
# prompt: เพิ่ม columns ใน future_df ที่วันที่ตรงกันของ ismarketday, isschoolday, holiday

# สร้าง DataFrame สำหรับ ismarketday, isschoolday, holiday จาก df ต้นฉบับ
# โดยใช้เฉพาะวันที่ที่อยู่ใน future_df index
#additional_features_df = df[['ismarketday', 'isschoolday', 'holiday']].reindex(future_df.index)

# รวม columns เหล่านี้เข้ากับ future_df
#future_df['ismarketday'] = additional_features_df['ismarketday']
#future_df['isschoolday'] = additional_features_df['isschoolday']
#future_df['holiday'] = additional_features_df['holiday']

print(future_df.shape)
future_df

In [ ]:
future_df = future_df.fillna(0)
print(future_df.shape)
future_df

In [ ]:
predictions_recursive = []
current_lags = last_known_lags.copy()

print("Starting 7-step recursive forecast...")
for i in range(7):
    calendar_features = future_df.iloc[i][['year', 'time_index']]

    input_vector = np.concatenate([current_lags, calendar_features.values]).reshape(1, -1)

    prediction = model_recursive.predict(input_vector)[0]
    predictions_recursive.append(prediction)

    current_lags = np.roll(current_lags, -1)
    current_lags[-1] = prediction
    print(f"Step {i+1}/7 predicted: {prediction:.2f}")

    print("\nRecursive forecasting completed.")

#Evaluate

In [ ]:
# ประเมินผลและแสดงกราฟ ---
actual_values = test_data[target].values
rmse_recursive = np.sqrt(mean_squared_error(actual_values, predictions_recursive))
mae_recursive = mean_absolute_error(actual_values, predictions_recursive)

print(f"\nRMSE of Recursive Forecast: {rmse_recursive:.2f}")
print(f"MAE of Recursive Forecast: {mae_recursive:.2f}")

plt.figure(figsize=(12, 7))
plt.plot(test_data.index, actual_values, 'o-', label='Actual Sales', color='blue')
plt.plot(test_data.index, predictions_recursive, 'x-', label='Recursive Forecast', color='red')
plt.title('7-Day Recursive Forecast vs Actual Sales', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Sales Sum')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# prompt: mae

mae_recursive

In [ ]:
import pickle

# Save the trained model
filename = 'Linear_Regression_model.pkl'
pickle.dump(model_recursive, open(filename, 'wb'))
print(f"Model saved as {filename}")

In [ ]:
# Load the saved model
loaded_model = pickle.load(open('Linear_Regression_model.pkl', 'rb'))
print("Model loaded successfully.")

# Get the last 7 lags from the original dataframe for prediction
last_7_lags = df[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']].tail(1).values.flatten()

# Prepare future dates and calendar features for the next 7 days
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7, freq='D')
future_df_predict = pd.DataFrame(index=future_dates)
future_df_predict['year'] = future_df_predict.index.year
future_df_predict['time_index'] = range(df['time_index'].max() + 1, df['time_index'].max() + 1 + 7)

# Make recursive predictions
predictions_recursive_loaded = []
current_lags_loaded = last_7_lags.copy()

print("Starting 7-step recursive forecast using loaded model...")
for i in range(7):
    calendar_features_loaded = future_df_predict.iloc[i][['year', 'time_index']]
    input_vector_loaded = np.concatenate([current_lags_loaded, calendar_features_loaded.values]).reshape(1, -1)

    prediction_loaded = loaded_model.predict(input_vector_loaded)[0]
    predictions_recursive_loaded.append(prediction_loaded)

    current_lags_loaded = np.roll(current_lags_loaded, -1)
    current_lags_loaded[-1] = prediction_loaded
    print(f"Step {i+1}/7 predicted: {prediction_loaded:.2f}")

print("\nRecursive forecasting completed using loaded model.")

# Display the predictions with dates
predictions_df = pd.DataFrame({'Date': future_dates, 'Predicted Sales': predictions_recursive_loaded})
predictions_df = predictions_df.set_index('Date')
print("\n7-Day Sales Predictions:")
display(predictions_df)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure df_original is loaded (assuming it's available from previous cells)
# If not, you might need to load it again
# df_original = pd.read_csv("/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv")
# df_original['date'] = pd.to_datetime(df_original['date'])
# df_original = df_original.set_index('date')

# Calculate the average sales_sum for each day of the week for the last 3 months
last_date_original = df_original.index.max()
three_months_ago = last_date_original - pd.DateOffset(months=3)
recent_df = df_original[df_original.index >= three_months_ago].copy() # Use copy to avoid SettingWithCopyWarning

average_sales_last_3_months = recent_df.groupby('day')['sales_sum'].mean().reindex(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])

# Ensure predictions_df is available (assuming it's available from previous cells)
# If not, you might need to regenerate predictions or load from a file
# predictions_df = ... # Load or generate your predictions_df here

# Plot the linear graph
plt.figure(figsize=(10, 6))
plt.plot(average_sales_last_3_months.index, average_sales_last_3_months.values, marker='o', linestyle='-', color='blue', label='Average Sales Last 3 Months')

# Add the predicted sales to the plot
# Extract day of the week from predicted dates
predicted_days_of_week = predictions_df.index.day_name()
plt.plot(predicted_days_of_week, predictions_df['Predicted Sales'].values, marker='x', linestyle='-', color='red', label='Predicted Sales (Next 7 Days)')

plt.xlabel('Day of the Week')
plt.ylabel('Sales Sum')
plt.ylim(0)
plt.title('Sales Sum by Day of the Week (Average Last 3 Months vs Predicted)')
plt.legend()
plt.grid(True)
plt.show()

# Task
Plot a line graph using `matplotlib` with 'Day of the Week' on the x-axis and 'Sales Sum' on the y-axis. The graph should display two lines: a blue line representing the average sales for each day of the week over the last 90 days from the dataframe loaded from "/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv", and a red line representing the predicted sales for the next 7 days from the `predictions_df`, aligning the predicted sales with the corresponding day of the week. Ensure the y-axis starts at 0 and include a title, axis labels, a legend, and a grid.

## Calculate average sales (last 90 days)

### Subtask:
Calculate the average sales for each day of the week using the last 90 days of the original data (`df_original`).


**Reasoning**:
Calculate the average sales for each day of the week using the last 90 days of the original data.



In [ ]:
# Determine the last date in the df_original DataFrame's index.
last_date_original = df_original.index.max()

# Calculate the date 90 days prior to the last date.
ninety_days_ago = last_date_original - pd.DateOffset(days=90)

# Filter the df_original DataFrame to include only the data within the last 90 days.
recent_df_90_days = df_original[df_original.index >= ninety_days_ago].copy()

# Group the filtered DataFrame by the 'dayofweek' and calculate the mean of the 'sales_sum'.
# Reindex to ensure the days are in order. dayofweek 0 is Monday, 6 is Sunday.
average_sales_last_90_days = recent_df_90_days.groupby(recent_df_90_days.index.dayofweek)['sales_sum'].mean()

# Map the dayofweek index to actual day names for plotting
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
average_sales_last_90_days.index = average_sales_last_90_days.index.map(lambda x: day_names[x])

# Reindex to ensure the days are in the desired order for plotting
average_sales_last_90_days = average_sales_last_90_days.reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

print("Average sales for each day of the week over the last 90 days:")
display(average_sales_last_90_days)

## Prepare predicted data

### Subtask:
Extract the day of the week and predicted sales from the `predictions_df`.


**Reasoning**:
Extract the day of the week and predicted sales from the predictions_df for plotting.



In [ ]:
predictions_df['day_of_week'] = predictions_df.index.day_name()
predicted_sales = predictions_df['Predicted Sales']
predicted_days_of_week = predictions_df['day_of_week']

print("Predicted Sales:")
display(predicted_sales)
print("\nPredicted Days of the Week:")
display(predicted_days_of_week)

## Plot data

### Subtask:
Create a line plot using `matplotlib` showing the average sales by day of the week and the predicted sales by day of the week on the same graph.


**Reasoning**:
Create a line plot using matplotlib showing the average sales by day of the week and the predicted sales by day of the week on the same graph, including labels, title, legend, and grid.



In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(average_sales_last_90_days.index, average_sales_last_90_days.values, marker='o', linestyle='-', color='blue', label='Average Sales Last 90 Days')
plt.plot(predicted_days_of_week, predicted_sales.values, marker='x', linestyle='-', color='red', label='Predicted Sales (Next 7 Days)')
plt.xlabel('Day of the Week')
plt.ylabel('Sales Sum')
plt.ylim(bottom=0)
plt.title('Sales Sum by Day of the Week (Average Last 90 Days vs Predicted)')
plt.legend()
plt.grid(True)
plt.show()

## Summary:

### Data Analysis Key Findings

*   The average sales for the last 90 days were calculated for each day of the week.
*   Predicted sales and their corresponding days of the week were extracted from the predictions data.
*   A line plot was generated comparing the average sales over the last 90 days by day of the week with the predicted sales for the next 7 days by day of the week.

### Insights or Next Steps

*   Analyze the generated plot to identify any patterns or discrepancies between historical average sales and predicted sales for each day of the week.
*   Investigate why there was no data for Saturdays in the last 90 days in the filtered dataset.
